In [98]:
import pandas as pd

In [99]:
data = pd.read_csv("adult.csv",header=0,index_col=False,
                  names=['age','workclass','fnlwgt','education','education-num',
                        'marital-status','occupation','relationship','race','gender',
                        'capital-gain','capital-loss','hours-per-week','native-country','income'])

In [100]:
data = data[['age','workclass','education','gender','hours-per-week','occupation','income']]

In [101]:
data.head()

,age,workclass,education,gender,hours-per-week,occupation,income
0,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
1,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
2,53,Private,11th,Male,40,Handlers-cleaners,<=50K
3,28,Private,Bachelors,Female,40,Prof-specialty,<=50K
4,37,Private,Masters,Female,40,Exec-managerial,<=50K


In [102]:
print(data.gender.value_counts())

 Male      21789
 Female    10771
Name: gender, dtype: int64


### One Hot Encoding
- get_dummies -> 범주형 encoding

In [103]:
data_dummies = pd.get_dummies(data)

In [104]:
display(data_dummies.head())

,age,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,occupation_ Machine-op-inspct,occupation_ Other-service,occupation_ Priv-house-serv,occupation_ Prof-specialty,occupation_ Protective-serv,occupation_ Sales,occupation_ Tech-support,occupation_ Transport-moving,income_ <=50K,income_ >50K
0,50,13,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,38,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,53,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,28,40,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,37,40,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [105]:
features = data_dummies.loc[:,'age':'occupation_Transport-moving']
# Numpy array 추출
X = features.values
y = data_dummies['income_>50K'].values
print("X.shape:{} y.shape:{}".format(X.shape,y.shape))

KeyError: 'occupation_Transport-moving'

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train,y_train)
print("Test score : {:.2f}".format(logreg.score(X_test,y_test)))

In [70]:
# 숫자와 문자 특성을 띈 DataFrame 생성
demo_df = pd.DataFrame({'숫자 특성':[0,1,2,1],
                        '범주형 특성':['양말','여우','양말','상자']})
display(demo_df)

,숫자 특성,범주형 특성
0,0,양말
1,1,여우
2,2,양말
3,1,상자


In [71]:
pd.get_dummies(demo_df) # 숫자 특성만 바뀌지 않음

,숫자 특성,범주형 특성_상자,범주형 특성_양말,범주형 특성_여우
0,0,0,1,0
1,1,0,0,1
2,2,0,1,0
3,1,1,0,0


In [73]:
# demo_df['숫자 특성'] = demo_df['숫자 특성'].astype(str)
display(pd.get_dummies(demo_df,columns=['숫자 특성','범주형 특성'])) # 열을 명시하면 모두 범주형으로 간주

,숫자 특성_0,숫자 특성_1,숫자 특성_2,범주형 특성_상자,범주형 특성_양말,범주형 특성_여우
0,1,0,0,0,1,0
1,0,1,0,0,0,1
2,0,0,1,0,1,0
3,0,1,0,1,0,0


#### OneHotEncoder / ColumnTransformer

In [78]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False) # Sparse Matrix가 아닌 Numpy Array return
print(ohe.fit_transform(demo_df))

[[1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 0. 1. 0.]
 [0. 1. 0. 1. 0. 0.]]


In [80]:
print(ohe.get_feature_names())

['x0_0' 'x0_1' 'x0_2' 'x1_상자' 'x1_양말' 'x1_여우']


In [81]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [82]:
ct = ColumnTransformer(
    [("scaling",StandardScaler(), ['age','hours-per-week']),
    ("onehot", OneHotEncoder(sparse=False),
    ['workclass','education','gender','occupation'])])

In [95]:
data_features.isnull().sum()

age                  0
workclass         2799
education            0
gender               0
hours-per-week       0
occupation        2809
dtype: int64

In [96]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# income 제외 모든열 추출
data_features = data.drop('income',axis=1)

X_train,X_test,y_train,y_test = train_test_split(
    data_features, data.income,random_state=0)

ct.fit(X_train)
# X_train_trans = ct.transform(X_train)
# print(X_train_trans.shape)

In [97]:
ct.fit(X_train)

ValueError: Input contains NaN